In [1]:
import pandas as pd


# CW

In [10]:
def read_res_for_c(c):
    dfs = []
    c_str = str(float(c))
    if c == 0:
        c_str = "0"
    for run_id in range(10):
        df = pd.read_csv(f"results/results_cw_c_{c_str}_run_{run_id}.csv")
        df['run_id'] = run_id
        dfs.append(df)
    return pd.concat(dfs, axis=0)

In [11]:
def read_enriched_c(c):
    df = read_res_for_c(c)
    df = df.loc[df['epoch'].isin([3,4,5])]
    res_df = df[['test_acc', 'mean_l2_of_adv_samples']].agg(['mean', 'std'])
    # .agg({
    #     'test_acc': ['mean', 'std'],
    #     'mean_l2_of_adv_samples': ['mean', 'std']
    # })
    res_df['c'] = c
    return res_df

In [12]:
all_c = [0, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0]
res = pd.concat([read_enriched_c(c) for c in all_c], axis=0)
res.head()

,test_acc,mean_l2_of_adv_samples,c
mean,95.122667,0.224685,0.0
std,2.483733,0.000224,0.0
mean,94.044667,0.309317,0.1
std,3.661752,0.083544,0.1
mean,92.914667,0.594921,0.3


In [13]:
%pip install plotly nbformat

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
res

,test_acc,mean_l2_of_adv_samples,c
mean,95.122667,0.224685,0.0
std,2.483733,0.000224,0.0
mean,94.044667,0.309317,0.1
std,3.661752,0.083544,0.1
mean,92.914667,0.594921,0.3
std,5.411959,0.249035,0.3
mean,87.642333,1.365607,1.0
std,11.485776,0.610036,1.0
mean,79.379000,3.304912,3.0
std,19.259040,1.416660,3.0


In [32]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(
        x=res.loc['mean', 'c'], y=res.loc['mean', 'test_acc'], 
        # error_y= {"type": "data",
        #           "array": res.loc['std', 'test_acc']},
        name="Accuracy"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        x=res.loc['mean', 'c'], y=res.loc['mean', 'mean_l2_of_adv_samples'],
        # error_y= {"type": "data",
        #           "array": res.loc['std', 'mean_l2_of_adv_samples']},
        name="Adv. examples L2 norm"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Carlini-Wagner effect of constant <b>c</b>. Results taken on 10 different seeds."
)

# Set x-axis title
fig.update_xaxes(title_text="C", type='log')

# Set y-axes titles
fig.update_yaxes(title_text="Accuracy", secondary_y=False)
fig.update_yaxes(title_text="adv. images L2 norm", secondary_y=True)

fig.show()

# BB NES

In [117]:
NUM_RUNS = 7
def read_res_for_att_epochs(att_epochs, bb = False):
    dfs = []
    att_epochs_str = str(int(att_epochs))
    for run_id in range(NUM_RUNS):
        if bb:
            df = pd.read_csv(f"results/results_pgd_blackbox_att_epochs_{att_epochs_str}_run_{run_id}.csv")
        else:
            df = pd.read_csv(f"results/results_pgd_whitebox_att_epochs_{att_epochs_str}_run_{run_id}.csv")
        df['run_id'] = run_id
        dfs.append(df)
    return pd.concat(dfs, axis=0)
    
def read_enriched_att_epochs(att_epochs, bb):
    df = read_res_for_att_epochs(att_epochs, bb)
    df = df.loc[df['epoch'].isin([4,5])]
    res_df = df[['test_acc', 'mean_l2_of_adv_samples']].agg(['mean', 'std'])
    # .agg({
    #     'test_acc': ['mean', 'std'],
    #     'mean_l2_of_adv_samples': ['mean', 'std']
    # })
    res_df['att_epochs'] = att_epochs
    return res_df

In [118]:
all_att_epochs = [1, 2, 4, 8, 16, 32, 64]
bb_res = pd.concat([read_enriched_att_epochs(att_epochs, bb=True) for att_epochs in all_att_epochs], axis=0)
wb_res = pd.concat([read_enriched_att_epochs(att_epochs, bb=False) for att_epochs in all_att_epochs], axis=0)
bb_res['model_access'] = "BlackBox"
wb_res['model_access'] = "WhiteBox"
res = pd.concat([bb_res, wb_res], axis=0)
res.sample(5)

,test_acc,mean_l2_of_adv_samples,att_epochs,model_access
mean,64.411429,7.685494,8,WhiteBox
mean,98.755714,2.085403,1,WhiteBox
std,0.209622,0.031521,2,BlackBox
std,13.757526,0.335400,32,BlackBox
mean,98.915000,4.701376,4,BlackBox


In [119]:
import plotly.express as px

px.line(
    res.loc['mean'],
    x='att_epochs',
    y='test_acc',
    color="model_access",
    log_x=True,
)

In [95]:
import plotly.express as px

px.line(
    res.loc['mean'],
    x='att_epochs',
    y='mean_l2_of_adv_samples',
    color="model_access",
    log_x=True,
)